In [1]:
import torch

torch.randn(10, 2)[:-1].shape

C:\Users\janba\python-dev\kernel_attention_TFT\.venv\Lib\site-packages\torch\_subclasses\functional_tensor.py:258: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


torch.Size([9, 2])

In [2]:
x = [torch.randn(64, 4, 192, 192) for _ in range(10)]

torch.cat(x, dim=0).shape

torch.Size([640, 4, 192, 192])

In [12]:
x = [torch.randn(32, 192, 8), torch.randn(32, 192, 8), torch.randn(32, 192, 8)]
torch.stack(x, dim=1).mean(dim=1).shape

torch.Size([32, 192, 8])

In [10]:
torch.randn(32, 1, 192, 8).squeeze(dim=1).shape

torch.Size([32, 192, 8, 1])

In [2]:
def get_attention_hparam_grid(attn_name):

    hranges = {
        'sdp': [{}],
        'lin': [{}],
        'exp': [
            {"p_norm_sim": 1, "p_norm_mag": 1, "include_magnitude": True},
            {"p_norm_sim": 2, "p_norm_mag": 1, "include_magnitude": True},
            {"p_norm_sim": 1, "p_norm_mag": 2, "include_magnitude": True},
            {"p_norm_sim": 1, "p_norm_mag": 2, "include_magnitude": False},
        ],
        'per': [
            {"period": 0.01, "p_norm": 2, "include_magnitude": True},
            {"period": 0.1, "p_norm": 2, "include_magnitude": True},
            {"period": 1, "p_norm": 2, "include_magnitude": True},
            {"period": 10, "p_norm": 2, "include_magnitude": True},
        ],
        'lp': [
            {"period": 0.01, "p_norm": 2, "include_magnitude": True},
            {"period": 0.1, "p_norm": 2, "include_magnitude": True},
            {"period": 1, "p_norm": 2, "include_magnitude": True},
            {"period": 10, "p_norm": 2, "include_magnitude": True},
        ],
        'rq': [
            {"alpha": 0.01, "p_norm": 2, "include_magnitude": True},
            {"alpha": 0.1, "p_norm": 2, "include_magnitude": True},
            {"alpha": 1, "p_norm": 2, "include_magnitude": True},
            {"alpha": 10, "p_norm": 2, "include_magnitude": True},
        ],
        'imp': [
            {"R_features": 64, "p_norm": 2, "include_magnitude": True},
            {"R_features": 128, "p_norm": 2, "include_magnitude": True},
        ],
        'cp': [
            # sim=2, mag=2
            {"period": 0.01, "p_norm_sim": 2, "p_norm_mag": 2, "include_magnitude": True},
            {"period": 0.1, "p_norm_sim": 2, "p_norm_mag": 2, "include_magnitude": True},
            {"period": 1, "p_norm_sim": 2, "p_norm_mag": 2, "include_magnitude": True},
            {"period": 10, "p_norm_sim": 2, "p_norm_mag": 2, "include_magnitude": True},
            # sim=1, mag=2
            {"period": 0.01, "p_norm_sim": 1, "p_norm_mag": 2, "include_magnitude": True},
            {"period": 0.1, "p_norm_sim": 1, "p_norm_mag": 2, "include_magnitude": True},
            {"period": 1, "p_norm_sim": 1, "p_norm_mag": 2, "include_magnitude": True},
            {"period": 10, "p_norm_sim": 1, "p_norm_mag": 2, "include_magnitude": True},
        ],
    }

    return hranges[attn_name]

In [6]:
from itertools import product

names = ['lin', 'per', 'exp', 'lp']

grids = [[(name, value) for value in get_attention_hparam_grid(name)] for name in names]
grids = list(product(*grids))

for params in grids:
    print(dict(params))

{'lin': {}, 'per': {'period': 0.01, 'p_norm': 2, 'include_magnitude': True}, 'exp': {'p_norm_sim': 1, 'p_norm_mag': 1, 'include_magnitude': True}, 'lp': {'period': 0.01, 'p_norm': 2, 'include_magnitude': True}}
{'lin': {}, 'per': {'period': 0.01, 'p_norm': 2, 'include_magnitude': True}, 'exp': {'p_norm_sim': 1, 'p_norm_mag': 1, 'include_magnitude': True}, 'lp': {'period': 0.1, 'p_norm': 2, 'include_magnitude': True}}
{'lin': {}, 'per': {'period': 0.01, 'p_norm': 2, 'include_magnitude': True}, 'exp': {'p_norm_sim': 1, 'p_norm_mag': 1, 'include_magnitude': True}, 'lp': {'period': 1, 'p_norm': 2, 'include_magnitude': True}}
{'lin': {}, 'per': {'period': 0.01, 'p_norm': 2, 'include_magnitude': True}, 'exp': {'p_norm_sim': 1, 'p_norm_mag': 1, 'include_magnitude': True}, 'lp': {'period': 10, 'p_norm': 2, 'include_magnitude': True}}
{'lin': {}, 'per': {'period': 0.01, 'p_norm': 2, 'include_magnitude': True}, 'exp': {'p_norm_sim': 2, 'p_norm_mag': 1, 'include_magnitude': True}, 'lp': {'period'

In [7]:
len(grids)

64

In [4]:
[[(name, value) for value in get_attention_hparam_grid(name)] for name in names]

[[('lin', {})],
 [('per', {'period': 0.01, 'p_norm': 2, 'include_magnitude': True}),
  ('per', {'period': 0.1, 'p_norm': 2, 'include_magnitude': True}),
  ('per', {'period': 1, 'p_norm': 2, 'include_magnitude': True}),
  ('per', {'period': 10, 'p_norm': 2, 'include_magnitude': True})],
 [('exp', {'p_norm_sim': 1, 'p_norm_mag': 1, 'include_magnitude': True}),
  ('exp', {'p_norm_sim': 2, 'p_norm_mag': 1, 'include_magnitude': True}),
  ('exp', {'p_norm_sim': 1, 'p_norm_mag': 2, 'include_magnitude': True}),
  ('exp', {'p_norm_sim': 1, 'p_norm_mag': 2, 'include_magnitude': False})]]

In [1]:
from modeling import InterpretableMultiHeadAttention, InterpretableMultiHeadAttention2

C:\Users\janba\python-dev\kernel_attention_TFT\.venv\Lib\site-packages\torch\_subclasses\functional_tensor.py:258: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [2]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [47]:
config = {
    "n_head": 4,
    "hidden_size": 128,
    "dropout": 0.1,
    "example_length": 192,
    "attn_dropout": 0.0,
    "batch_size": 1
}
config = dotdict(config)

In [48]:
import torch

x = torch.randn(config.batch_size, config.example_length, config.hidden_size)

In [75]:
import attention as ATTN

nH = config.n_head
t = config.example_length
nD = config.hidden_size // config.n_head

qk = torch.randn(64, t, nH, nD)
v = torch.randn(64, t, nD)

qk.shape, v.shape

(torch.Size([64, 192, 4, 32]), torch.Size([64, 192, 32]))

In [86]:
y, attn_w = ATTN.DotProductAttention()(qk.transpose(1, 2), qk.transpose(1, 2), v.unsqueeze(1))
y, attn_w = ATTN.ChangePointKernelAttention()(qk.transpose(1, 2), qk.transpose(1, 2), v.unsqueeze(1))

y.shape, attn_w.shape

(torch.Size([64, 4, 192, 32]), torch.Size([64, 4, 192, 192]))

In [51]:
imha = InterpretableMultiHeadAttention(config, ATTN.DotProductAttention(0.0))
imha2 = InterpretableMultiHeadAttention2(config)

imha.eval()
imha2.eval()

for name, p in imha.named_parameters():
    torch.nn.init.ones_(p)

for name, p in imha2.named_parameters():
    torch.nn.init.ones_(p)

In [74]:
with torch.no_grad():
    y, attn_w = imha(x)
y.shape, attn_w.shape

(torch.Size([1, 192, 128]), torch.Size([1, 4, 192, 192]))

In [73]:
with torch.no_grad():
    y2, attn_w2 = imha2(x)
y2.shape, attn_w2.shape

(torch.Size([1, 192, 128]), torch.Size([1, 4, 192, 192]))